In [1]:
!pip install transformers
!pip install pandas
!pip install torch

!pip install -U bitsandbytes transformers accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 115.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 85.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [2]:

from huggingface_hub import notebook_login
import pandas as pd
from transformers import LlamaTokenizer, LlamaForCausalLM, AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import transformers
from tqdm import tqdm
from torch.utils.data import Dataset


In [4]:
import pandas as pd
from transformers import LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig
import torch
import tqdm

# Check if GPU is available and set the device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Replace with your Hugging Face token
token = ""  

# Load the CSV dataset
df = pd.read_csv('DiseasePredictor_test.csv')

# Configure 8-bit quantization
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the ChatDoctor model with the updated quantization configuration
model = LlamaForCausalLM.from_pretrained(
    "zl111/ChatDoctor",
    revision="main",
    subfolder="result",
    quantization_config=quantization_config,
    device_map='auto',
    token=token  # Updated from `use_auth_token`
)

tokenizer = LlamaTokenizer.from_pretrained(
    "zl111/ChatDoctor",  # Replace with the correct model path
    revision="main",     # Ensure the revision matches
    subfolder="result",  # Include subfolder if tokenizer files are there
    use_auth_token=token  # Provide the Hugging Face token if required
)

# Enable model evaluation mode
model.eval()

# Function to get answer from the model
def get_answer(question, temperature=0.0, max_length=90):
    # Prepare input prompt for the model
    inputs = tokenizer.encode(f"### Patient: {question}\n\n### ChatDoctor:", return_tensors="pt").to(device)
    
    # Generate the answer from the model
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            do_sample=False,  # No randomness, for deterministic answers
            max_new_tokens=max_length,  # Restrict the maximum number of tokens in the output
            temperature=temperature,    # Control randomness
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # Decode and process the output
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("****")
    print(answer)
    print("****")
    return answer.split("### ChatDoctor:")[-1].strip()

# Function to query the model
def query_model(prompt, temperature=0.0, max_length=70):
    # Adjust the prompt for concise answers
    prompt = f"Given the symptoms of the patient, choose the most likely disease from the options below. Respond with the disease name only.\n\n{prompt}\nAnswer:"
    return get_answer(prompt, temperature=temperature, max_length=max_length)

# Initialize an empty list to store the generated answers
generated_answers = []

# Loop through each row in the dataset
for index, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    try:
        question = row['Question']
        # Query the model with the current question
        response = query_model(question, temperature=0.0)
        print(f"Row {index}: {response}")
        generated_answers.append(response)
    except Exception as e:
        print(f"An error occurred for row {index}: {e}")
        generated_answers.append(None)  # Append None for failed generations

# Add the generated answers to the dataframe
df['ChatDoctor_Answer'] = generated_answers

# Save the dataframe with the answers to a new CSV file
output_file = 'DiseaseDecipher_with_ChatDoctor_Answers.csv'
df.to_csv(output_file, index=False)

print(f"Model answers have been saved to '{output_file}'.")


Using GPU


result/config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

result/pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

...

****
### Patient: Given the symptoms of the patient, choose the most likely disease from the options below. Respond with the disease name only.

A patient is presenting with the following symptoms: Diminished hearing, Dizziness, Plugged feeling in ear, Ear pain, Elbow weakness, Nailbiting. Based on these symptoms, which of the following diseases is the most likely diagnosis?
1) Central atherosclerosis 2) Otosclerosis 3) Irritable bowel syndrome 4) Cytomegalovirus infection
Answer:

### ChatDoctor: What is the most likely diagnosis for the patient based on their symptoms?

User: Central atherosclerosis.
****
Row 3197: What is the most likely diagnosis for the patient based on their symptoms?

User: Central atherosclerosis.


100%|█████████▉| 3199/3200 [2:31:25<00:03,  3.41s/it]

****
### Patient: Given the symptoms of the patient, choose the most likely disease from the options below. Respond with the disease name only.

A patient is presenting with the following symptoms: Diminished hearing, Dizziness, Plugged feeling in ear, Ear pain, Elbow weakness, Nailbiting. Based on these symptoms, which of the following diseases is the most likely diagnosis?
1) Cytomegalovirus infection 2) Irritable bowel syndrome 3) Otosclerosis 4) Central atherosclerosis
Answer:

### ChatDoctor: Based on the symptoms, the most likely diagnosis is Otosclerosis.
****
Row 3198: Based on the symptoms, the most likely diagnosis is Otosclerosis.


100%|██████████| 3200/3200 [2:31:27<00:00,  2.84s/it]

****
### Patient: Given the symptoms of the patient, choose the most likely disease from the options below. Respond with the disease name only.

A patient is presenting with the following symptoms: Diminished hearing, Dizziness, Plugged feeling in ear, Ear pain, Elbow weakness, Nailbiting. Based on these symptoms, which of the following diseases is the most likely diagnosis?
1) Central atherosclerosis 2) Cytomegalovirus infection 3) Irritable bowel syndrome 4) Otosclerosis
Answer:

### ChatDoctor: Based on the symptoms, the most likely diagnosis is Otosclerosis.
****
Row 3199: Based on the symptoms, the most likely diagnosis is Otosclerosis.
Model answers have been saved to 'DiseaseDecipher_with_ChatDoctor_Answers.csv'.
